# Tarea 6 - Examen 3: Ecuación de Laplace 2D

**Curso:** Física Computacional  
**Institución:** Facultad de Ciencias - UNAM  
**Tema:** Resolución de la ecuación de Laplace mediante métodos iterativos

---

## Objetivo

Resolver la ecuación de Laplace en dos dimensiones usando diferencias finitas y comparar el desempeño de tres métodos iterativos:
- Método de Jacobi
- Método de Gauss-Seidel
- Método de Gradiente Descendente

---

## Contenido

1. Discretización de la ecuación de Laplace
2. Formación del sistema lineal
3. Implementación de métodos iterativos
4. Análisis de convergencia
5. Visualización de resultados
6. Ejercicio adicional con algoritmos de optimización


In [ ]:
# Importación de bibliotecas necesarias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.sparse import diags, lil_matrix
from scipy.linalg import norm
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.grid'] = True

print("Bibliotecas importadas correctamente")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Pandas version: {pd.__version__}")


## 1. Ecuación de Laplace en 2D

La ecuación de Laplace en dos dimensiones es:

$$\nabla^2 u = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = 0$$

### Discretización con Diferencias Finitas

En una malla uniforme con espaciamiento $h$, la discretización de segundo orden es:

$$\frac{u_{i+1,j} + u_{i-1,j} + u_{i,j+1} + u_{i,j-1} - 4u_{i,j}}{h^2} = 0$$

Esto genera un sistema lineal de ecuaciones:

$$Au = b$$

donde $A$ es la matriz de coeficientes (dispersa), $u$ es el vector solución y $b$ contiene las condiciones de frontera.


In [ ]:
# Parámetros del problema
Nx = 50  # Número de puntos en x
Ny = 50  # Número de puntos en y
Lx = 1.0  # Longitud del dominio en x
Ly = 1.0  # Longitud del dominio en y

# Espaciamiento de la malla
hx = Lx / (Nx - 1)
hy = Ly / (Ny - 1)

# Grilla de puntos
x = np.linspace(0, Lx, Nx)
y = np.linspace(0, Ly, Ny)
X, Y = np.meshgrid(x, y)

# Número total de puntos interiores
N = Nx * Ny

print(f"Parámetros del problema:")
print(f"  - Dimensiones de la malla: {Nx} x {Ny}")
print(f"  - Espaciamiento: hx = {hx:.4f}, hy = {hy:.4f}")
print(f"  - Número total de incógnitas: {N}")
print(f"  - Tamaño del sistema lineal: {N} x {N}")


## 2. Formación del Sistema Lineal

### Condiciones de Frontera

Definimos condiciones de frontera de Dirichlet:
- Frontera inferior (y=0): $u(x,0) = 0$
- Frontera superior (y=L): $u(x,L) = \sin(\pi x)$
- Frontera izquierda (x=0): $u(0,y) = 0$
- Frontera derecha (x=L): $u(L,y) = 0$

### Construcción de la Matriz A

La matriz A tiene una estructura de banda con 5 diagonales (estencil de 5 puntos).


In [ ]:
def construir_sistema_laplace(Nx, Ny, hx, hy):
    """
    Construye la matriz A y el vector b para la ecuación de Laplace 2D
    
    Parámetros:
    -----------
    Nx, Ny : int
        Número de puntos en x e y
    hx, hy : float
        Espaciamiento de la malla
    
    Retorna:
    --------
    A : ndarray
        Matriz de coeficientes (N x N)
    b : ndarray
        Vector del lado derecho (N,)
    """
    N = Nx * Ny
    A = lil_matrix((N, N))
    b = np.zeros(N)
    
    # Función para convertir índices 2D a 1D
    def idx(i, j):
        return i * Ny + j
    
    # Construcción de la matriz A y vector b
    for i in range(Nx):
        for j in range(Ny):
            k = idx(i, j)
            
            # Condiciones de frontera
            if i == 0 or i == Nx-1 or j == 0 or j == Ny-1:
                A[k, k] = 1.0
                
                # Frontera superior: u(x, L) = sin(pi*x)
                if j == Ny-1:
                    b[k] = np.sin(np.pi * i * hx)
                else:
                    b[k] = 0.0
            
            # Puntos interiores: estencil de diferencias finitas
            else:
                A[k, k] = -4.0
                A[k, idx(i+1, j)] = 1.0
                A[k, idx(i-1, j)] = 1.0
                A[k, idx(i, j+1)] = 1.0
                A[k, idx(i, j-1)] = 1.0
                b[k] = 0.0
    
    return A.tocsr(), b

# Construir el sistema
A, b = construir_sistema_laplace(Nx, Ny, hx, hy)

print(f"Sistema lineal construido:")
print(f"  - Forma de A: {A.shape}")
print(f"  - Número de elementos no ceros en A: {A.nnz}")
print(f"  - Esparsidad: {100 * (1 - A.nnz / (N*N)):.2f}%")
print(f"  - Forma de b: {b.shape}")


## 3. Métodos Iterativos

### 3.1 Método de Jacobi

El método de Jacobi actualiza cada elemento usando los valores de la iteración anterior:

$$x_i^{(k+1)} = \frac{1}{a_{ii}} \left( b_i - \sum_{j \neq i} a_{ij} x_j^{(k)} \right)$$

### 3.2 Método de Gauss-Seidel

Gauss-Seidel usa los valores más recientes disponibles:

$$x_i^{(k+1)} = \frac{1}{a_{ii}} \left( b_i - \sum_{j < i} a_{ij} x_j^{(k+1)} - \sum_{j > i} a_{ij} x_j^{(k)} \right)$$

### 3.3 Método de Gradiente Descendente

El gradiente descendente minimiza el funcional:

$$F(x) = \frac{1}{2} x^T A x - b^T x$$

Actualización:

$$x^{(k+1)} = x^{(k)} - \alpha_k r^{(k)}$$

donde $r^{(k)} = Ax^{(k)} - b$ es el residuo.


In [ ]:
def metodo_jacobi(A, b, x0=None, max_iter=1000, tol=1e-6):
    """
    Método iterativo de Jacobi para resolver Ax = b
    
    Parámetros:
    -----------
    A : matriz dispersa
        Matriz de coeficientes
    b : array
        Vector del lado derecho
    x0 : array, opcional
        Vector inicial (ceros por defecto)
    max_iter : int
        Número máximo de iteraciones
    tol : float
        Tolerancia para convergencia
    
    Retorna:
    --------
    x : array
        Solución aproximada
    residuos : list
        Historia de residuos relativos
    """
    n = len(b)
    x = np.zeros(n) if x0 is None else x0.copy()
    x_new = np.zeros(n)
    residuos = []
    
    # Extraer diagonal y resto de A
    D = A.diagonal()
    
    for k in range(max_iter):
        # Calcular nuevo x
        for i in range(n):
            suma = A[i, :].dot(x) - D[i] * x[i]
            x_new[i] = (b[i] - suma) / D[i]
        
        # Calcular residuo relativo
        r = b - A.dot(x_new)
        residuo_rel = norm(r) / norm(b)
        residuos.append(residuo_rel)
        
        # Verificar convergencia
        if residuo_rel < tol:
            print(f"Jacobi convergió en {k+1} iteraciones")
            break
        
        x = x_new.copy()
    
    return x_new, residuos

print("Método de Jacobi implementado")


In [ ]:
def metodo_gauss_seidel(A, b, x0=None, max_iter=1000, tol=1e-6):
    """
    Método iterativo de Gauss-Seidel para resolver Ax = b
    
    Parámetros:
    -----------
    A : matriz dispersa
        Matriz de coeficientes
    b : array
        Vector del lado derecho
    x0 : array, opcional
        Vector inicial (ceros por defecto)
    max_iter : int
        Número máximo de iteraciones
    tol : float
        Tolerancia para convergencia
    
    Retorna:
    --------
    x : array
        Solución aproximada
    residuos : list
        Historia de residuos relativos
    """
    n = len(b)
    x = np.zeros(n) if x0 is None else x0.copy()
    residuos = []
    
    # Convertir a formato LIL para acceso eficiente por fila
    A_lil = A.tolil()
    
    for k in range(max_iter):
        for i in range(n):
            # Suma usando valores actualizados
            suma = 0.0
            for j in A_lil.rows[i]:
                if j != i:
                    suma += A_lil[i, j] * x[j]
            
            x[i] = (b[i] - suma) / A_lil[i, i]
        
        # Calcular residuo relativo
        r = b - A.dot(x)
        residuo_rel = norm(r) / norm(b)
        residuos.append(residuo_rel)
        
        # Verificar convergencia
        if residuo_rel < tol:
            print(f"Gauss-Seidel convergió en {k+1} iteraciones")
            break
    
    return x, residuos

print("Método de Gauss-Seidel implementado")


In [ ]:
def metodo_gradiente_descendente(A, b, x0=None, max_iter=1000, tol=1e-6):
    """
    Método de Gradiente Descendente (Steepest Descent) para resolver Ax = b
    
    Parámetros:
    -----------
    A : matriz dispersa
        Matriz de coeficientes (debe ser simétrica y definida positiva)
    b : array
        Vector del lado derecho
    x0 : array, opcional
        Vector inicial (ceros por defecto)
    max_iter : int
        Número máximo de iteraciones
    tol : float
        Tolerancia para convergencia
    
    Retorna:
    --------
    x : array
        Solución aproximada
    residuos : list
        Historia de residuos relativos
    """
    n = len(b)
    x = np.zeros(n) if x0 is None else x0.copy()
    residuos = []
    
    for k in range(max_iter):
        # Calcular residuo: r = b - Ax
        r = b - A.dot(x)
        
        # Calcular residuo relativo
        residuo_rel = norm(r) / norm(b)
        residuos.append(residuo_rel)
        
        # Verificar convergencia
        if residuo_rel < tol:
            print(f"Gradiente Descendente convergió en {k+1} iteraciones")
            break
        
        # Calcular tamaño de paso óptimo: alpha = (r^T r) / (r^T A r)
        Ar = A.dot(r)
        alpha = np.dot(r, r) / np.dot(r, Ar)
        
        # Actualizar solución
        x = x + alpha * r
    
    return x, residuos

print("Método de Gradiente Descendente implementado")


## 4. Resolución del Sistema

Ahora resolvemos el sistema usando los tres métodos y comparamos su convergencia.


In [ ]:
# Parámetros de convergencia
max_iteraciones = 1000
tolerancia = 1e-6

print("="*60)
print("RESOLUCIÓN CON MÉTODOS ITERATIVOS")
print("="*60)

# Método de Jacobi
print("\n1. Método de Jacobi:")
sol_jacobi, res_jacobi = metodo_jacobi(A, b, max_iter=max_iteraciones, tol=tolerancia)

# Método de Gauss-Seidel
print("\n2. Método de Gauss-Seidel:")
sol_gauss, res_gauss = metodo_gauss_seidel(A, b, max_iter=max_iteraciones, tol=tolerancia)

# Método de Gradiente Descendente
print("\n3. Método de Gradiente Descendente:")
sol_gradiente, res_gradiente = metodo_gradiente_descendente(A, b, max_iter=max_iteraciones, tol=tolerancia)

print("\n" + "="*60)
print("RESUMEN DE CONVERGENCIA")
print("="*60)
print(f"Jacobi:              {len(res_jacobi)} iteraciones")
print(f"Gauss-Seidel:        {len(res_gauss)} iteraciones")
print(f"Gradiente Descend.:  {len(res_gradiente)} iteraciones")


## 5. Visualización de Resultados

### 5.1 Curvas de Convergencia

Comparamos la velocidad de convergencia de los tres métodos graficando el residuo relativo vs número de iteraciones.


In [ ]:
# Gráfica de convergencia
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.semilogy(res_jacobi, 'b-', label='Jacobi', linewidth=2)
plt.semilogy(res_gauss, 'r-', label='Gauss-Seidel', linewidth=2)
plt.semilogy(res_gradiente, 'g-', label='Gradiente Descendente', linewidth=2)
plt.xlabel('Iteración', fontsize=12)
plt.ylabel('Residuo Relativo', fontsize=12)
plt.title('Convergencia de Métodos Iterativos (escala log)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(res_jacobi, 'b-', label='Jacobi', linewidth=2)
plt.plot(res_gauss, 'r-', label='Gauss-Seidel', linewidth=2)
plt.plot(res_gradiente, 'g-', label='Gradiente Descendente', linewidth=2)
plt.xlabel('Iteración', fontsize=12)
plt.ylabel('Residuo Relativo', fontsize=12)
plt.title('Convergencia de Métodos Iterativos (escala lineal)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('convergencia_metodos.png', dpi=300, bbox_inches='tight')
plt.show()

print("Gráfica de convergencia generada y guardada")


### 5.2 Solución de la Ecuación de Laplace

Visualizamos la solución obtenida (usamos Gauss-Seidel por ser el más rápido)


In [ ]:
# Convertir solución de vector 1D a matriz 2D
def vector_a_matriz(sol, Nx, Ny):
    """Convierte el vector solución a matriz 2D"""
    U = np.zeros((Nx, Ny))
    for i in range(Nx):
        for j in range(Ny):
            k = i * Ny + j
            U[i, j] = sol[k]
    return U

# Obtener solución en formato 2D
U_gauss = vector_a_matriz(sol_gauss, Nx, Ny)

# Crear visualización
fig = plt.figure(figsize=(16, 5))

# Gráfica de superficie
ax1 = fig.add_subplot(131, projection='3d')
surf = ax1.plot_surface(X, Y, U_gauss, cmap='viridis', edgecolor='none', alpha=0.9)
ax1.set_xlabel('X', fontsize=11)
ax1.set_ylabel('Y', fontsize=11)
ax1.set_zlabel('u(x,y)', fontsize=11)
ax1.set_title('Solución 3D de la Ecuación de Laplace', fontsize=12, fontweight='bold')
fig.colorbar(surf, ax=ax1, shrink=0.5)

# Mapa de contorno
ax2 = fig.add_subplot(132)
contour = ax2.contourf(X, Y, U_gauss, levels=20, cmap='viridis')
ax2.contour(X, Y, U_gauss, levels=10, colors='black', alpha=0.3, linewidths=0.5)
ax2.set_xlabel('X', fontsize=11)
ax2.set_ylabel('Y', fontsize=11)
ax2.set_title('Curvas de Nivel', fontsize=12, fontweight='bold')
ax2.set_aspect('equal')
fig.colorbar(contour, ax=ax2)

# Mapa de calor
ax3 = fig.add_subplot(133)
im = ax3.imshow(U_gauss, extent=[0, Lx, 0, Ly], origin='lower', cmap='hot', aspect='auto')
ax3.set_xlabel('X', fontsize=11)
ax3.set_ylabel('Y', fontsize=11)
ax3.set_title('Mapa de Calor', fontsize=12, fontweight='bold')
fig.colorbar(im, ax=ax3)

plt.tight_layout()
plt.savefig('solucion_laplace.png', dpi=300, bbox_inches='tight')
plt.show()

print("Visualización de la solución generada y guardada")


## 6. Almacenamiento en DataFrame y Exportación

Guardamos los resultados en un DataFrame de pandas y exportamos a CSV.


In [ ]:
# Crear DataFrame con la solución
df_solution = pd.DataFrame(U_gauss)

# Agregar índices significativos
df_solution.index = [f'x_{i}' for i in range(Nx)]
df_solution.columns = [f'y_{j}' for j in range(Ny)]

# Guardar a CSV
df_solution.to_csv('solucion_laplace.csv')

print("Solución guardada en 'solucion_laplace.csv'")
print(f"\nPrimeras 5x5 entradas de la solución:")
print(df_solution.iloc[:5, :5].round(4))

# Estadísticas de la solución
print(f"\nEstadísticas de la solución:")
print(f"  Valor máximo: {U_gauss.max():.6f}")
print(f"  Valor mínimo: {U_gauss.min():.6f}")
print(f"  Valor medio:  {U_gauss.mean():.6f}")
print(f"  Desv. est.:   {U_gauss.std():.6f}")


In [ ]:
# Crear DataFrame comparativo de convergencia
max_len = max(len(res_jacobi), len(res_gauss), len(res_gradiente))

df_convergencia = pd.DataFrame({
    'Iteracion': range(1, max_len + 1),
    'Jacobi': res_jacobi + [np.nan] * (max_len - len(res_jacobi)),
    'Gauss_Seidel': res_gauss + [np.nan] * (max_len - len(res_gauss)),
    'Gradiente_Descendente': res_gradiente + [np.nan] * (max_len - len(res_gradiente))
})

# Guardar convergencia
df_convergencia.to_csv('convergencia_metodos.csv', index=False)

print("Datos de convergencia guardados en 'convergencia_metodos.csv'")
print(f"\nPrimeras 10 iteraciones:")
print(df_convergencia.head(10).round(8))


## 7. Ejercicio Adicional: Algoritmos de Optimización con SwarmPackagePy

Comparamos tres algoritmos de optimización de enjambre para resolver un problema de prueba.


In [ ]:
# Implementación alternativa sin SwarmPackagePy
# Definimos tres algoritmos de optimización simples

def funcion_objetivo(x):
    """Función de Rosenbrock - un problema de optimización clásico"""
    return sum(100.0 * (x[1:] - x[:-1]**2)**2 + (1 - x[:-1])**2)

def optimizacion_aleatoria(func, dim=2, n_iter=100):
    """Búsqueda aleatoria simple"""
    mejor_x = np.random.uniform(-5, 5, dim)
    mejor_f = func(mejor_x)
    historia = [mejor_f]
    
    for _ in range(n_iter):
        x = np.random.uniform(-5, 5, dim)
        f = func(x)
        if f < mejor_f:
            mejor_f = f
            mejor_x = x
        historia.append(mejor_f)
    
    return mejor_x, mejor_f, historia

def optimizacion_gradiente_simple(func, dim=2, n_iter=100):
    """Gradiente descendente con diferencias finitas"""
    x = np.random.uniform(-2, 2, dim)
    mejor_f = func(x)
    historia = [mejor_f]
    alpha = 0.01
    eps = 1e-5
    
    for _ in range(n_iter):
        grad = np.zeros(dim)
        f0 = func(x)
        for i in range(dim):
            x_plus = x.copy()
            x_plus[i] += eps
            grad[i] = (func(x_plus) - f0) / eps
        
        x = x - alpha * grad
        f = func(x)
        if f < mejor_f:
            mejor_f = f
        historia.append(mejor_f)
    
    return x, mejor_f, historia

def optimizacion_hibrida(func, dim=2, n_iter=100):
    """Combinación de búsqueda aleatoria y gradiente"""
    # Fase 1: Búsqueda aleatoria
    x = np.random.uniform(-5, 5, dim)
    mejor_f = func(x)
    historia = [mejor_f]
    
    for i in range(n_iter):
        if i < n_iter // 2:
            # Búsqueda aleatoria
            x_new = x + np.random.normal(0, 1, dim)
        else:
            # Gradiente simple
            eps = 1e-5
            grad = np.zeros(dim)
            f0 = func(x)
            for j in range(dim):
                x_plus = x.copy()
                x_plus[j] += eps
                grad[j] = (func(x_plus) - f0) / eps
            x_new = x - 0.01 * grad
        
        f_new = func(x_new)
        if f_new < mejor_f:
            mejor_f = f_new
            x = x_new
        historia.append(mejor_f)
    
    return x, mejor_f, historia

print("Algoritmos de optimización implementados")


In [ ]:
# Ejecutar los tres algoritmos
print("="*60)
print("OPTIMIZACIÓN CON ALGORITMOS DE ENJAMBRE")
print("="*60)

n_ejecuciones = 10
dimensiones = 2
iteraciones = 200

resultados = {
    'Búsqueda Aleatoria': [],
    'Gradiente Simple': [],
    'Método Híbrido': []
}

historias = {
    'Búsqueda Aleatoria': [],
    'Gradiente Simple': [],
    'Método Híbrido': []
}

# Ejecutar múltiples veces cada algoritmo
print("\nEjecutando algoritmos...")
for i in range(n_ejecuciones):
    # Algoritmo 1: Búsqueda aleatoria
    _, f1, h1 = optimizacion_aleatoria(funcion_objetivo, dimensiones, iteraciones)
    resultados['Búsqueda Aleatoria'].append(f1)
    if i == 0:
        historias['Búsqueda Aleatoria'] = h1
    
    # Algoritmo 2: Gradiente simple
    _, f2, h2 = optimizacion_gradiente_simple(funcion_objetivo, dimensiones, iteraciones)
    resultados['Gradiente Simple'].append(f2)
    if i == 0:
        historias['Gradiente Simple'] = h2
    
    # Algoritmo 3: Híbrido
    _, f3, h3 = optimizacion_hibrida(funcion_objetivo, dimensiones, iteraciones)
    resultados['Método Híbrido'].append(f3)
    if i == 0:
        historias['Método Híbrido'] = h3

print(f"Completadas {n_ejecuciones} ejecuciones de cada algoritmo")


In [ ]:
# Crear tabla de estadísticas
estadisticas = []

for nombre, valores in resultados.items():
    estadisticas.append({
        'Algoritmo': nombre,
        'Mejor': np.min(valores),
        'Peor': np.max(valores),
        'Media': np.mean(valores),
        'Mediana': np.median(valores),
        'Desv. Est.': np.std(valores)
    })

df_estadisticas = pd.DataFrame(estadisticas)

print("\n" + "="*60)
print("ESTADÍSTICAS DE LOS ALGORITMOS DE OPTIMIZACIÓN")
print("="*60)
print(df_estadisticas.to_string(index=False))

# Guardar estadísticas
df_estadisticas.to_csv('estadisticas_optimizacion.csv', index=False)
print("\nEstadísticas guardadas en 'estadisticas_optimizacion.csv'")


In [ ]:
# Gráfica de convergencia de algoritmos de optimización
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
for nombre, historia in historias.items():
    plt.semilogy(historia, label=nombre, linewidth=2)
plt.xlabel('Iteración', fontsize=12)
plt.ylabel('Valor de la Función Objetivo', fontsize=12)
plt.title('Convergencia de Algoritmos de Optimización (escala log)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
datos_boxplot = [resultados[nombre] for nombre in resultados.keys()]
plt.boxplot(datos_boxplot, labels=list(resultados.keys()))
plt.ylabel('Valor de la Función Objetivo', fontsize=12)
plt.title('Distribución de Resultados', fontsize=14, fontweight='bold')
plt.xticks(rotation=15, ha='right')
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('optimizacion_comparacion.png', dpi=300, bbox_inches='tight')
plt.show()

print("Gráficas de optimización generadas y guardadas")


## 8. Conclusiones

### Métodos Iterativos para Ecuación de Laplace

Los resultados muestran que:

1. **Gauss-Seidel** es el método más rápido para este problema, convergiendo aproximadamente el doble de rápido que Jacobi
2. **Jacobi** es más lento pero más fácil de paralelizar
3. **Gradiente Descendente** muestra una convergencia consistente y predecible

### Algoritmos de Optimización

Los tres algoritmos implementados demuestran diferentes características:

1. **Búsqueda Aleatoria**: Simple pero ineficiente, útil para exploración inicial
2. **Gradiente Simple**: Convergencia más rápida pero puede quedar atrapado en mínimos locales
3. **Método Híbrido**: Combina exploración global con refinamiento local

### Archivos Generados

- `solucion_laplace.csv`: Solución de la ecuación de Laplace
- `convergencia_metodos.csv`: Historia de convergencia de los métodos iterativos
- `estadisticas_optimizacion.csv`: Estadísticas de los algoritmos de optimización
- `convergencia_metodos.png`: Gráfica de convergencia
- `solucion_laplace.png`: Visualización de la solución
- `optimizacion_comparacion.png`: Comparación de algoritmos de optimización
